In [1]:
pip install openpyxl

     |████████████████████████████████| 243 kB 4.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')


df_ampla_raw = pd.read_excel ("ampla_raw_data.xlsx",engine='openpyxl',sheet_name='Sheet1')
df_powerbi_raw = pd.read_excel('powerbi_raw_data.xlsx',engine='openpyxl',sheet_name='Sheet1')


#df_working_hours = df_powerbi_raw[df_powerbi_raw['engine_status']=='On']

# clean ampla data 

df_ampla_1 = df_ampla_raw [['Start Time','End Time','Duration','Equipment Id','Classification','Comments','Cause']]
df_ampla_1= df_ampla_1.dropna().reset_index()
df_ampla_1['Start Time']= pd.to_datetime(df_ampla_1['Start Time'])
df_ampla_1['End Time'] = pd.to_datetime(df_ampla_1['End Time'])
df_ampla_1 = df_ampla_1.sort_values(by=['Equipment Id','Start Time'],ascending=[True, True])
df_ampla_1 = df_ampla_1.reset_index(drop=True)
df_ampla_1['Duration_1']= (df_ampla_1['End Time']-df_ampla_1['Start Time'])
df_ampla_1['Duration_in_minutes']= df_ampla_1['Duration_1'].dt.total_seconds()//60
df_ampla_1 = df_ampla_1[df_ampla_1['Duration_in_minutes']>1]
df_ampla_1['Duplication_within_6_hours']=0

# within 6 hours duplication filter for unplanned work 
df_ampla_1 = df_ampla_1[(df_ampla_1['Classification']=='Unscheduled Downtime (UD)')]
df_ampla_1 = df_ampla_1.reset_index(drop=True)


for i in range (1,len(df_ampla_1.axes[0])):

    if (abs(df_ampla_1['Start Time'][i] - df_ampla_1['Start Time'][i-1])/np.timedelta64(1, 'h')) <6 :
        df_ampla_1['Duplication_within_6_hours'][i] = 1
           
#print(df_ampla_1.head(40))
    
#slice data set for non duplicates and unplanned work to get breakdown count

df_ampla_filtered = df_ampla_1[(df_ampla_1['Duplication_within_6_hours']==0)]
df_ampla_filtered['year'] = pd.DatetimeIndex(df_ampla_filtered['Start Time']).year
df_ampla_filtered['month'] = pd.DatetimeIndex(df_ampla_filtered['Start Time']).month

df_ampla_breakdown_counts = df_ampla_filtered.groupby(['year','month','Equipment Id']).size().reset_index(name='counts')
df_ampla_breakdown_counts['Last_four_digits'] = df_ampla_breakdown_counts['Equipment Id'].str[-4:]

#print(df_ampla_breakdown_counts.head(40))
  
#filter powerbi data to get worked hours 

df_powerbi_ready = df_powerbi_raw[df_powerbi_raw['engine_status'] == 'On']
df_powerbi_worked_hours = df_powerbi_ready.groupby(['time - Year','time - Month','equipment_name']).sum().reset_index()
df_powerbi_worked_hours['equipment_name'] = df_powerbi_worked_hours['equipment_name'].astype(str)
df_powerbi_worked_hours['Last_four_digits'] = df_powerbi_worked_hours['equipment_name'].str[-4:]
d = {'January':1, 'February':2, 'March':3, 'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,
    'December':12}

df_powerbi_worked_hours['time - Month'] = df_powerbi_worked_hours['time - Month'].map(d)


# merge data sets &calc MTBF
df_merged = pd.merge(df_ampla_breakdown_counts, df_powerbi_worked_hours, how="left", left_on=['year','month',"Last_four_digits"],
                     right_on=['time - Year','time - Month','Last_four_digits'])
df_merged = df_merged.dropna()
df_merged["group"] = df_merged['Equipment Id'].str[0:5]

#print(df_merged.head(40))

df_mtbf = df_merged.groupby(['year','month','group']).sum().reset_index()
df_mtbf['MTBF'] = df_mtbf['Sum of hours']/df_mtbf['counts']



# Last updated time

In [7]:
print(df_ampla_1['Start Time'].sort_values().iloc[-1])

2021-08-26 09:30:16


# MTBF trend from Jan-2020 

In [14]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

options =df_mtbf['group'].astype("category").cat.categories.tolist()
def plot_func(freq):
    
    df_loader_mtbf = df_mtbf[df_mtbf['group']==freq].reset_index(drop=True)
    df_loader_mtbf['DATE'] = pd.to_datetime(df_loader_mtbf[['year', 'month']].assign(DAY=28))
    

    x=df_loader_mtbf['DATE']
    y=df_loader_mtbf['MTBF']
    
    plt.figure(figsize=(12,8), dpi=80)
    plt.plot(x, y)
    for i,j in zip(x,y):
        plt.annotate(int(j),xy=(i,j+1))
        
    plt.ylabel("MTBF(Hours)")
    plt.xlabel('Time')
    plt.xticks(x)
    plt.title('MTBF trend of '+ freq)
    plt.xticks(rotation=90)
    plt.ylim(0)
    plt.show()
    

interact(plot_func, freq = widgets.Dropdown(options=options,value=options[6],description='Machine type:',disabled=False))

interactive(children=(Dropdown(description='Machine type:', index=6, options=('DOZER', 'DRILL', 'EXCAV', 'GRAD…

<function __main__.plot_func(freq)>

# Monthly MTBF by equipment ID

In [15]:
year_options = df_merged['year'].unique()
machine_options =df_mtbf['group'].astype("category").cat.categories.tolist()

def plot_trucks_func(year,month,machine):
       
    
    df_merged['MTBF'] = df_merged['Sum of hours']/df_merged['counts']
    df_year = df_merged[df_merged['year']==year]
    df_year_month = df_year[df_year['month']==month]
    df_trucks = df_year_month[df_year_month['group']==machine]
    df_trucks = df_trucks.sort_values(by=['MTBF'])
    
    if month in df_year['month'].unique():
        average_truck_mtbf = df_mtbf.loc[(df_mtbf['year']==year) & (df_mtbf['group']==machine)& (df_mtbf['month']==month)]
        average_truck_mtbf = average_truck_mtbf.iloc[0]['MTBF']

        plt.figure(figsize=(10, 8), dpi=80)
        plt.barh(df_trucks['Equipment Id'],df_trucks['MTBF'])

        plt.ylabel(machine)
        plt.xlabel('MTBF(Hours)')
        plt.xticks(list(plt.xticks()[0]) + [average_truck_mtbf])
        plt.axvline(x=average_truck_mtbf , linestyle='--')
        plt.title('MTBF of each ' +machine)
        plt.show()
        
    else:
        print('INVALID MONTH !!')
        

interact(plot_trucks_func, year = widgets.Dropdown(options=year_options,value=2021,description='Year:',disabled=False),
        month=widgets.Dropdown(options=[1,2,3,4,5,6,7,8,9,10,11,12],value=6,description='Month:',disabled=False),
        machine=widgets.Dropdown(options=machine_options,value='TRUCK',description='Machine type:',disabled=False))

interactive(children=(Dropdown(description='Year:', index=1, options=(2020, 2021), value=2021), Dropdown(descr…

<function __main__.plot_trucks_func(year, month, machine)>

# Overall MTBF by equipment ID

In [16]:
import numpy 

year_options = df_merged['year'].unique()
year_options = list(numpy.append(year_options,'All'))
machine_options =df_mtbf['group'].astype("category").cat.categories.tolist()

def plot_yearly_mtbf_func(year,machine):
    
    if year== 'All':
    
        df_merged_1 = df_merged.groupby(['group','Equipment Id']).sum().reset_index()
        df_merged_1['MTBF_year'] = df_merged_1['Sum of hours']/df_merged_1['counts']

        df_trucks = df_merged_1[df_merged_1['group']==machine]
        df_trucks = df_trucks.sort_values(by=['MTBF_year'])

        plt.figure(figsize=(10, 8), dpi=80)
        plt.barh(df_trucks['Equipment Id'],df_trucks['MTBF_year'])

        plt.ylabel(machine)
        plt.xlabel('MTBF(Hours)')
        plt.xticks(list(plt.xticks()[0]))
        plt.title('MTBF of each ' +machine)
        plt.show()
  
    
    
    
    else:
    
        year = int(year)
        df_merged_1 = df_merged.groupby(['year','group','Equipment Id']).sum().reset_index()
        df_merged_1['MTBF_year'] = df_merged_1['Sum of hours']/df_merged_1['counts']

        df_year = df_merged_1[df_merged_1['year']==year]
        df_trucks = df_year[df_year['group']==machine]
        df_trucks = df_trucks.sort_values(by=['MTBF_year'])

        plt.figure(figsize=(10, 8), dpi=80)
        plt.barh(df_trucks['Equipment Id'],df_trucks['MTBF_year'])

        plt.ylabel(machine)
        plt.xlabel('MTBF(Hours)')
        plt.xticks(list(plt.xticks()[0]))
        plt.title('MTBF of each ' +machine)
        plt.show()

        

interact(plot_yearly_mtbf_func, year = widgets.Dropdown(options=year_options,value='2021',description='Year:',disabled=False),
         machine=widgets.Dropdown(options=machine_options,value='TRUCK',description='Machine type:',disabled=False))

interactive(children=(Dropdown(description='Year:', index=1, options=('2020', '2021', 'All'), value='2021'), D…

<function __main__.plot_yearly_mtbf_func(year, machine)>

# Breakdown count by system

In [36]:
df_ampla_filtered['Equipment group']=df_ampla_filtered['Equipment Id'].str[0:5]
df_system = df_ampla_filtered.groupby(['year','month','Equipment group','Cause']).size().reset_index(name='Cause_counts')

year_options_systems = df_system['year'].unique()
def plot_systems_func(year,month,machine):
       
    df_year = df_system[df_system['year']==year]
    df_year_month = df_year[df_year['month']==month]
    df_machine = df_year_month[df_year_month['Equipment group']==machine]
    df_machine = df_machine.sort_values(by=['Cause_counts'],ascending=False)
      
    if month in df_year['month'].unique():

        plt.figure(figsize=(10, 8), dpi=80)
        plt.bar(df_machine['Cause'],df_machine['Cause_counts'])

        plt.ylabel('Fault count')
        plt.xlabel('Filure cause')
        plt.xticks(rotation=90)
        plt.title('Failure causes in month ' +str(month))
        plt.show()
        
    else:
        print('INVALID MONTH !!')
        

interact(plot_systems_func, year = widgets.Dropdown(options=year_options_systems,value=2021,description='Year:',disabled=False),
        month=widgets.Dropdown(options=[1,2,3,4,5,6,7,8,9,10,11,12],value=6,description='Month:',disabled=False),
         machine=widgets.Dropdown(options=machine_options,value='TRUCK',description='Machine type:',disabled=False)
        )

interactive(children=(Dropdown(description='Year:', index=2, options=(2019, 2020, 2021), value=2021), Dropdown…

<function __main__.plot_systems_func(year, month, machine)>

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')